<a href="https://colab.research.google.com/github/RicottaChz/Pytorch_Youtube/blob/main/cifar10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# transformer, attention, universial approximation thm

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms

In [ ]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4,
                                          shuffle=True, num_workers=2)
testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

100%|██████████| 170498071/170498071 [00:10<00:00, 15637356.22it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [ ]:
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
class Model(nn.Module):
  def __init__(self):
    super().__init__()
    self.conv1 = nn.Conv2d(3, 6, 5)
    self.pool = nn.MaxPool2d(2, 2)
    self.conv2 = nn.Conv2d(6, 16, 5)
    self.fc1 = nn.Linear(16*5*5, 120)
    self.fc2 = nn.Linear(120, 84)
    self.out = nn.Linear(84, 10)

  def forward(self, x):
    x = self.pool(F.relu(self.conv1(x)))
    x = self.pool(F.relu(self.conv2(x)))
    x = x.view(-1, 16*5*5)
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = self.out(x)
    return x

In [ ]:
model = Model()

In [ ]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
epochs = 5

for epoch in range(epochs):
  losses = 0.0
  for i, data in enumerate(trainloader, 0):
    inputs, labels = data

    outputs = model(inputs)

    loss = criterion(outputs, labels.long())
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if i % 2000 == 1999:
      print(f'Epoch: {epoch + 1}, Batch: {i + 1}, Loss: {loss.item()}')
      losses = 0.0


Epoch: 1, Batch: 2000, Loss: 1.0681833028793335
Epoch: 1, Batch: 4000, Loss: 2.0445024967193604
Epoch: 1, Batch: 6000, Loss: 1.339385747909546
Epoch: 1, Batch: 8000, Loss: 1.6299145221710205
Epoch: 1, Batch: 10000, Loss: 0.784845232963562
Epoch: 1, Batch: 12000, Loss: 1.3392927646636963
Epoch: 2, Batch: 2000, Loss: 1.4874615669250488
Epoch: 2, Batch: 4000, Loss: 0.6602450013160706
Epoch: 2, Batch: 6000, Loss: 0.6687097549438477
Epoch: 2, Batch: 8000, Loss: 0.4656561315059662
Epoch: 2, Batch: 10000, Loss: 1.2549257278442383
Epoch: 2, Batch: 12000, Loss: 1.2345223426818848
Epoch: 3, Batch: 2000, Loss: 1.745337963104248
Epoch: 3, Batch: 4000, Loss: 1.568719744682312
Epoch: 3, Batch: 6000, Loss: 1.2978622913360596
Epoch: 3, Batch: 8000, Loss: 2.176421642303467
Epoch: 3, Batch: 10000, Loss: 1.2184562683105469
Epoch: 3, Batch: 12000, Loss: 1.2376353740692139
Epoch: 4, Batch: 2000, Loss: 0.8429399132728577
Epoch: 4, Batch: 4000, Loss: 1.1839860677719116
Epoch: 4, Batch: 6000, Loss: 1.56352174

In [ ]:
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))

Accuracy of the network on the 10000 test images: 59 %


In [ ]:
class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        c = (predicted == labels).squeeze()
        for i in range(4):
            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1


for i in range(10):
    print('Accuracy of %5s : %2d %%' % (
        classes[i], 100 * class_correct[i] / class_total[i]))

Accuracy of plane : 63 %
Accuracy of   car : 65 %
Accuracy of  bird : 46 %
Accuracy of   cat : 56 %
Accuracy of  deer : 51 %
Accuracy of   dog : 35 %
Accuracy of  frog : 55 %
Accuracy of horse : 70 %
Accuracy of  ship : 83 %
Accuracy of truck : 71 %
